In [364]:
from elasticsearch import Elasticsearch
from datetime import datetime, timedelta

import json
cluster = ['ip-172-31-0-107', 'ip-172-31-0-100', 'ip-172-31-0-105', 'ip-172-31-0-106']
es = Elasticsearch(cluster, port=9200)

In [365]:
def dispatch(ctime, location, driver, name, p1=None, p2=None):
        #d = retrieveDriver(driver, es)
        
        p = scanPassenger(location, es)
        
        dDoc = {"ctime": ctime, "location": location, "status": "idle"}
        if p:
            doc = {"status": "pickup", "driver": driver, "ctime": ctime}
            updatePassenger(p['id'], doc, es)

            if p1: 
                dDoc['p2'] = p['id']
            else:
                dDoc['p1'] = p['id']
                
            dDoc['status'] = "pickup"
            dDoc['destination'] = p['location']
            dDoc['altdest1'] = p['id']
            

        updateDriver(driver, dDoc, es)
        
        bulk = (1, '{{doc: {}}}'.format(json.dumps(dDoc)))
        return (bulk)

In [ ]:
      'driver': {'type': 'string', "include_in_all": "false"},
        'destination': {'type': 'geo_point', 'lat_lon': 'true', "include_in_all": "false"},
        'destinationid':{'type': 'string', "include_in_all": "false"},                
        'altdest1': {'type': 'geo_point', 'lat_lon': 'true', "include_in_all": "false"},
        'altdestid1':{'type': 'string', "include_in_all": "false"},                    
        'altdest2': {'type': 'geo_point', 'lat_lon': 'true', "include_in_all": "false"},
        'altdestid2':{'type': 'string', "include_in_all": "false"}, 
        'origin': {'type': 'geo_point', 'lat_lon': 'true', "include_in_all": "false"},
        'path': {'type': 'string', "include_in_all": "false"}


In [507]:
window = 'now-2h'
location =[40.7488,-73.9854]
q =  { "from" : 0, "size" : 1,
                 "query": {
              "filtered": {
                "query" : {
                 "term" : {"status": "wait"}},
                "filter": {
                    "geo_distance": {
                        "distance": '3km',
                        "distance_type": "plane", 
                        "location": location }}
            }},
              "sort": [{
 "_geo_distance": {
      "location": location,
         "order": "asc",
          "unit": "km", 
 "distance_type": "plane" 
                  }}],
         
    }

res = es.search(index='passenger', doc_type='rolling', body=q, ignore=[404, 400])



In [483]:
def retrieveDriver(driver, es):
    _ = es.get(index='driver', doc_type='rolling', id=driver, ignore=[404, 400])
    return _
  

In [494]:
shoulds = []
for i in ["Empire Building", "Times Square","Intrepid Sea, Air & Space Museum", "Metropolitan Opera House", "American Museum of Natural History"]:
    shoulds.append({'match': {'destinationid': i}})
    shoulds.append({'match': {'altdest1id': i}}) 
    shoulds.append({'match': {'altdest2id': i}}) 

In [508]:
res

{u'_shards': {u'failed': 0, u'successful': 20, u'total': 20},
 u'hits': {u'hits': [{u'_id': u'18204',
    u'_index': u'passenger',
    u'_score': None,
    u'_source': {u'altdest1': [40.7488, -73.9854],
     u'altdest1id': u'Empire Building',
     u'altdest2': [40.7588, -73.9851],
     u'altdest2id': u'Times Square',
     u'city': u'NYC',
     u'ctime': u'2016-10-09T20:49:40.613952Z',
     u'destination': [40.7644, -73.9993],
     u'destinationid': u'Intrepid Sea, Air & Space Museum',
     u'id': 18204,
     u'location': [40.7514, -73.9896],
     u'name': u'passenger_18204',
     u'path': [[40.7514, -73.9896]],
     u'status': u'wait'},
    u'_type': u'rolling',
    u'sort': [0.5498713962723231]}],
  u'max_score': None,
  u'total': 3192},
 u'timed_out': False,
 u'took': 60}